In [1]:
import pandas as pd
import dagster as dg
import numpy as num
import duckdb as db
import shopify as shop
import requests as req
import os
from datetime import datetime
from dotenv import load_dotenv

In [4]:
os.getcwd()

'/Users/ryan_burton/shopify-dagster-duckdb'

In [2]:
os.chdir('..')

In [11]:
subfolder_name = "duckdb_database"

In [16]:
os.chdir("dagster_project")

In [12]:
con = db.connect("database.db")

In [13]:
tables_info_df = con.execute("SELECT * FROM duckdb_tables").fetchdf()
tables_info_df = pd.DataFrame(tables_info_df)

In [14]:
tables_info_df

,database_name,database_oid,schema_name,schema_oid,table_name,table_oid,comment,tags,internal,temporary,has_primary_key,estimated_size,column_count,index_count,check_constraint_count,sql


In [9]:
def test_dataframe_not_empty(df):
    """
    Tests if a given Pandas DataFrame is not empty.

    Args:
        df (pd.DataFrame): The DataFrame to test.

    Returns:
        bool: True if the DataFrame is not empty, False otherwise.
    """
    return not df.empty

In [10]:
test_dataframe_not_empty(tables_info_df)

True

In [7]:
load_dotenv()

shop_url = os.getenv("SHOPIFY_API_URL")
api_version = os.getenv("SHOPIFY_API_VERSION")
api_key = os.getenv("SHOPIFY_API_KEY")

headers = {
    "X-Shopify-Access-Token": api_key,
    "Content-Type": "application/json"
}

In [9]:
products_url = f"https://no-name-business-123.myshopify.com/admin/api/2025-07/products.json"
response = req.get(products_url, headers=headers)
products_data = response.json().get("products", [])
products_df = pd.DataFrame(products_data)

In [13]:
products_df['snapshot_date'] = datetime.now()

In [2]:
os.chdir('..')
subfolder_name = "duckdb_database"
os.chdir(subfolder_name)

In [17]:
os.getcwd()

'/Users/ryan_burton/shopify-dagster-duckdb/dagster_project'

In [3]:
con = db.connect("database.db")

In [4]:
con.execute("SELECT * FROM duckdb_tables").fetchdf()

,database_name,database_oid,schema_name,schema_oid,table_name,table_oid,comment,tags,internal,temporary,has_primary_key,estimated_size,column_count,index_count,check_constraint_count,sql
0,database,570,entities,574,shopify_products,576,None,{},False,False,True,34,19,1,0,CREATE TABLE entities.shopify_products(id BIGI...
1,database,570,main,572,ducks,585,None,{},False,False,False,1,2,0,0,"CREATE TABLE ducks(age INTEGER, breed VARCHAR);"
2,database,570,main,572,penguins,591,None,{},False,False,False,1,2,0,0,"CREATE TABLE penguins(age INTEGER, breed VARCH..."
3,database,570,prep,597,animals,599,None,{},False,False,False,2,3,0,0,"CREATE TABLE prep.animals(age INTEGER, breed V..."


In [18]:
entities_schema_name = "entities"
con.execute("CREATE SCHEMA IF NOT EXISTS entities_schema_name;")

In [19]:
con.execute("CREATE TABLE IF NOT EXISTS entities.shopify_products AS SELECT * FROM products_df;")

In [20]:
con.execute("INSERT OR REPLACE INTO entities.shopify_products SELECT * FROM products_df;")

BinderException: Binder Error: There are no UNIQUE/PRIMARY KEY Indexes that refer to this table, ON CONFLICT is a no-op

In [22]:
products_df.head()

,id,title,body_html,vendor,product_type,created_at,handle,updated_at,published_at,template_suffix,published_scope,tags,status,admin_graphql_api_id,variants,options,images,image,snapshot_date
0,7691993874534,Gift Card,This is a gift card for the store,Snowboard Vendor,giftcard,2025-07-17T18:32:27-04:00,gift-card,2025-07-18T07:52:51-04:00,2025-07-17T18:32:28-04:00,None,web,,active,gid://shopify/Product/7691993874534,"[{'id': 42947791093862, 'product_id': 76919938...","[{'id': 9829888295014, 'product_id': 769199387...","[{'id': 33697401667686, 'alt': 'Gift card that...","{'id': 33697401667686, 'alt': 'Gift card that ...",2025-07-18 12:39:25.026789
1,7691994136678,Selling Plans Ski Wax,None,No Name Business 123,accessories,2025-07-17T18:32:27-04:00,selling-plans-ski-wax,2025-07-18T08:25:12-04:00,2025-07-17T18:32:28-04:00,None,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994136678,"[{'id': 42947791683686, 'product_id': 76919941...","[{'id': 9829888557158, 'product_id': 769199413...","[{'id': 33697402159206, 'alt': 'A bar of golde...","{'id': 33697402159206, 'alt': 'A bar of golden...",2025-07-18 12:39:25.026789
2,7691994333286,The 3p Fulfilled Snowboard,None,No Name Business 123,snowboard,2025-07-17T18:32:28-04:00,the-3p-fulfilled-snowboard,2025-07-18T08:52:13-04:00,2025-07-17T18:32:29-04:00,None,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994333286,"[{'id': 42947791847526, 'product_id': 76919943...","[{'id': 9829888753766, 'product_id': 769199433...","[{'id': 33697402126438, 'alt': 'Top and bottom...","{'id': 33697402126438, 'alt': 'Top and bottom ...",2025-07-18 12:39:25.026789
3,7691994038374,The Archived Snowboard,None,Snowboard Vendor,snowboard,2025-07-17T18:32:27-04:00,the-archived-snowboard,2025-07-18T08:25:12-04:00,None,None,web,"Archived, Premium, Snow, Snowboard, Sport, Winter",archived,gid://shopify/Product/7691994038374,"[{'id': 42947791356006, 'product_id': 76919940...","[{'id': 9829888458854, 'product_id': 769199403...","[{'id': 33697401602150, 'alt': 'The top and bo...","{'id': 33697401602150, 'alt': 'The top and bot...",2025-07-18 12:39:25.026789
4,7691994234982,The Collection Snowboard: Hydrogen,None,Hydrogen Vendor,snowboard,2025-07-17T18:32:28-04:00,the-collection-snowboard-hydrogen,2025-07-18T13:07:16-04:00,2025-07-17T18:32:29-04:00,None,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994234982,"[{'id': 42947791552614, 'product_id': 76919942...","[{'id': 9829888655462, 'product_id': 769199423...","[{'id': 33697401765990, 'alt': 'Top and bottom...","{'id': 33697401765990, 'alt': 'Top and bottom ...",2025-07-18 12:39:25.026789


In [33]:
con.execute(f"CREATE TABLE IF NOT EXISTS entities.shopify_products AS SELECT * FROM products_df;")

In [34]:
con.execute(f"ALTER TABLE entities.shopify_products ADD PRIMARY KEY(id, snapshot_date);")

In [35]:
con.execute(f"INSERT OR REPLACE INTO entities.shopify_products SELECT * FROM products_df;")

In [5]:
con.execute(f"SELECT * FROM entities.shopify_products").fetchdf()

,id,title,body_html,vendor,product_type,created_at,handle,updated_at,published_at,template_suffix,published_scope,tags,status,admin_graphql_api_id,variants,options,images,image,snapshot_date
0,7691993874534,Gift Card,This is a gift card for the store,Snowboard Vendor,giftcard,2025-07-17T18:32:27-04:00,gift-card,2025-07-18T07:52:51-04:00,2025-07-17T18:32:28-04:00,<NA>,web,,active,gid://shopify/Product/7691993874534,"[{'id': 42947791093862, 'product_id': 76919938...","[{'id': 9829888295014, 'product_id': 769199387...","[{'id': 33697401667686, 'alt': 'Gift card that...","{'id': 33697401667686, 'alt': 'Gift card that ...",2025-07-18 13:03:12.524867
1,7691994136678,Selling Plans Ski Wax,None,No Name Business 123,accessories,2025-07-17T18:32:27-04:00,selling-plans-ski-wax,2025-07-18T08:25:12-04:00,2025-07-17T18:32:28-04:00,<NA>,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994136678,"[{'id': 42947791683686, 'product_id': 76919941...","[{'id': 9829888557158, 'product_id': 769199413...","[{'id': 33697402159206, 'alt': 'A bar of golde...","{'id': 33697402159206, 'alt': 'A bar of golden...",2025-07-18 13:03:12.524867
2,7691994333286,The 3p Fulfilled Snowboard,None,No Name Business 123,snowboard,2025-07-17T18:32:28-04:00,the-3p-fulfilled-snowboard,2025-07-18T08:52:13-04:00,2025-07-17T18:32:29-04:00,<NA>,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994333286,"[{'id': 42947791847526, 'product_id': 76919943...","[{'id': 9829888753766, 'product_id': 769199433...","[{'id': 33697402126438, 'alt': 'Top and bottom...","{'id': 33697402126438, 'alt': 'Top and bottom ...",2025-07-18 13:03:12.524867
3,7691994038374,The Archived Snowboard,None,Snowboard Vendor,snowboard,2025-07-17T18:32:27-04:00,the-archived-snowboard,2025-07-18T08:25:12-04:00,None,<NA>,web,"Archived, Premium, Snow, Snowboard, Sport, Winter",archived,gid://shopify/Product/7691994038374,"[{'id': 42947791356006, 'product_id': 76919940...","[{'id': 9829888458854, 'product_id': 769199403...","[{'id': 33697401602150, 'alt': 'The top and bo...","{'id': 33697401602150, 'alt': 'The top and bot...",2025-07-18 13:03:12.524867
4,7691994234982,The Collection Snowboard: Hydrogen,None,Hydrogen Vendor,snowboard,2025-07-17T18:32:28-04:00,the-collection-snowboard-hydrogen,2025-07-18T13:07:16-04:00,2025-07-17T18:32:29-04:00,<NA>,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994234982,"[{'id': 42947791552614, 'product_id': 76919942...","[{'id': 9829888655462, 'product_id': 769199423...","[{'id': 33697401765990, 'alt': 'Top and bottom...","{'id': 33697401765990, 'alt': 'Top and bottom ...",2025-07-18 13:03:12.524867
5,7691994398822,The Collection Snowboard: Liquid,None,Hydrogen Vendor,snowboard,2025-07-17T18:32:28-04:00,the-collection-snowboard-liquid,2025-07-18T06:56:38-04:00,2025-07-17T18:32:29-04:00,<NA>,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994398822,"[{'id': 42947791913062, 'product_id': 76919943...","[{'id': 9829888819302, 'product_id': 769199439...","[{'id': 33697402388582, 'alt': 'Top and bottom...","{'id': 33697402388582, 'alt': 'Top and bottom ...",2025-07-18 13:03:12.524867
6,7691994300518,The Collection Snowboard: Oxygen,None,Hydrogen Vendor,snowboard,2025-07-17T18:32:28-04:00,the-collection-snowboard-oxygen,2025-07-18T08:47:52-04:00,2025-07-17T18:32:29-04:00,<NA>,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691994300518,"[{'id': 42947791814758, 'product_id': 76919943...","[{'id': 9829888720998, 'product_id': 769199430...","[{'id': 33697402191974, 'alt': 'Top and bottom...","{'id': 33697402191974, 'alt': 'Top and bottom ...",2025-07-18 13:03:12.524867
7,7691993940070,The Compare at Price Snowboard,None,No Name Business 123,snowboard,2025-07-17T18:32:27-04:00,the-compare-at-price-snowboard,2025-07-18T07:40:24-04:00,2025-07-17T18:32:28-04:00,<NA>,web,"Accessory, Sport, Winter",active,gid://shopify/Product/7691993940070,"[{'id': 42947791323238, 'product_id': 76919939...","[{'id': 9829888360550, 'pro

In [22]:
con.execute("DROP TABLE entities.shopify_products;")